In [1]:
%run "C:\Users\o_thanakrit.b\OneDrive - KASIKORNBANKGROUP\เอกสาร\code_and_finding\template.ipynb"

In [2]:
fy2023 = dd.read_parquet(mart_fy2023_path/"data_fy2023_fix_mkt_exp_add_usage_consu_pri_crd_blk_rm_wlx_reFormula.parquet")
ecl = dd.read_parquet(mart_fy2023_path/"profit"/"rolled_annual_card_ecl_delta_dec23vsdec22.parquet")
card_profit_net_ecl = dd.read_parquet(mart_fy2023_path/"profit"/"rolled_annual_card_profit_net_ecl.parquet")

card_os = dd.read_parquet(mart_fy2023_path/"rolled_annual_card_os.parquet")
card_cl = dd.read_parquet(mart_fy2023_path/"rolled_annual_card_cl.parquet")
card_util = dd.read_parquet(mart_fy2023_path/"rolled_annual_card_util_new.parquet").reset_index()
card_seg = dd.read_parquet(mart_fy2023_path/"rolled_annual_card_segment_adj.parquet").loc[:, ["uniq_key", "card_status_flag", "card_annual_segment_adj"]]

card_type = dd.read_parquet(staging_path/"card_type.parquet")
card_dim = dd.read_parquet(staging_path/"cust_card_dim.parquet")
tier_typ_gr = dd.read_csv("card_tier_type_mapper.csv")

# Use customer CL since all customer shared CL
cust_cl = dd.read_parquet(mart_fy2023_path/"rolled_annual_cust_cl.parquet")

# New SA SE
new_occ = dd.read_parquet(staging_path/"sa_se_bsn2.parquet").drop_duplicates()

In [3]:
new_occ.columns

Index(['uniq_key', 'sa_se_fnl', 'bsn_tp', 'bsn_dsc'], dtype='object')

## Focus segment analysis
- Affluent AUM - Revolver Freq.
- Affluent Income - Revolver Freq.
- Mass - Revolver Freq.

In [4]:
card_cube = dd.read_parquet(mart_fy2023_path/"card_cube"/"rolled_annual_card.parquet")

In [5]:
card_cube["uniq_key"].nunique().compute()

3997489

In [6]:
card_cube["cc_cst_real"].nunique().compute()

2329585

In [7]:
card_cube.columns

Index(['cc_cst_real', 'uniq_key', 'revenue', 'cost', 'cashback', 'cof',
       'fixed_cost', 'fraud_loss', 'reward_burn', 'scheme_fee', 'annual_fee',
       'cash_adv_fee', 'collection_fee', 'fx_fee', 'interchg_fee', 'int_inc',
       'mftp_total', 'activity_based_costing_expense', 'debt_collection_exp',
       'fee_waived_off', 'marketing_exp', 'specific_business_tax',
       'fee_paid_visa', 'issuer_license_fee', 'total_account_card_fee',
       'total_concierge_fee', 'total_issuer_domestic', 'total_issuer_inter',
       'gain_and_loss', 'interchange_fee', 'merchant_discount_fee',
       'net_interest_rec', 'past_due_amt', 'stmt_amt', 'mth_end_bal',
       'crn_pymt_due_amt', 'pymt_amt', 'cash_draw', 'smc_draw', 'spd_draw',
       'smp_draw', 'usage', 'ecl_delta', 'profit', 'profit_net_ecl',
       'card_status_flag', 'card_annual_segment_adj', 'card_tp',
       'issur_card_lvl2', 'monthly_avg_os_stmt_amt',
       'monthly_avg_os_mth_end_bal', 'monthly_avg_cl', 'annual_stmt_amt',
   

In [8]:
card_cube["Peer"].unique().compute()

0               Mass
1       Affluent AUM
2            Retails
3          Insurance
0           institue
0             Health
0    Affluent Income
1           Cashback
2             Online
Name: Peer, dtype: string

In [9]:
card_cube["card_annual_segment_adj"].unique().compute()

0                Paydown
0             Transactor
1    Revolver-Occasional
0               Inactive
1             Delinquent
2      Revolver-Frequent
Name: card_annual_segment_adj, dtype: string

In [10]:
focus_peer = ["Affluent AUM", "Affluent Income", "Mass"]
focus_behv = ["Revolver-Frequent"]

In [11]:
focus_cube = card_cube.loc[ (card_cube["Peer"].isin(focus_peer)) & (card_cube["card_annual_segment_adj"].isin(focus_behv)), :]

In [12]:
focus_cube.groupby(["Peer", "card_annual_segment_adj"]).agg(n_account = ("uniq_key", "count")).compute()

,,n_account
Peer,card_annual_segment_adj,
Mass,Revolver-Frequent,544312
Affluent Income,Revolver-Frequent,45981
Affluent AUM,Revolver-Frequent,10605


### SA/SE, Industry

In [13]:
card_dim.isna().sum().compute()

cc_cst_real          0
uniq_key             0
opn_dt               1
incm_tier       214086
sa_se_fnl            0
bsn_tp         2698017
bsn_dsc        2698017
dtype: int64

In [14]:
new_occ.isna().sum().compute()

uniq_key          0
sa_se_fnl         0
bsn_tp       527579
bsn_dsc      527630
dtype: int64

In [15]:
card_dim["uniq_key"].nunique().compute()

5266262

In [16]:
new_occ["uniq_key"].nunique().compute()

5266283

In [17]:
new_occ["uniq_key"].count().compute()

5266283

In [63]:
(focus_cube
 .merge(new_occ, on="uniq_key", how="left")
 .fillna({'sa_se_fnl':"NA", 'bsn_dsc':"NA"})
 .groupby(["Peer", "card_annual_segment_adj", "sa_se_fnl", "bsn_dsc"])
 .agg(n_account = ("uniq_key", "count"))
).compute().reset_index().to_clipboard()

### Utilization

In [51]:
focus_cube_util = focus_cube.loc[:, ["uniq_key", "Peer", "card_annual_segment_adj", 
                                     "annual_util_mth_end_bal", "monthly_avg_os_mth_end_bal", "monthly_avg_cl"]].compute()

In [53]:
(focus_cube_util
 .replace({"annual_util_mth_end_bal": {np.Inf: None, -np.Inf: None}})
 .dropna(subset=["annual_util_mth_end_bal"])
 .assign(annual_util_mth_end_bal_round = lambda x : x["annual_util_mth_end_bal"].astype(float).round(3))
 .groupby(["Peer", "card_annual_segment_adj"])
 .agg(n_account = ("uniq_key", "count"),
      avg_monthly_mth_end_bal = ("monthly_avg_os_mth_end_bal", "mean"),
      avg_monthly_cl = ("monthly_avg_cl", "mean"),
      avg_util = ("annual_util_mth_end_bal", "mean"), 
      mode_util = ("annual_util_mth_end_bal_round", lambda x: x.value_counts().index[0]), 
      mode_util_n = ("annual_util_mth_end_bal_round", lambda x: x.value_counts()[0]), 
      p1_util = ("annual_util_mth_end_bal", lambda x: np.percentile(x, q=1)),
      p10_util = ("annual_util_mth_end_bal", lambda x: np.percentile(x, q=10)),
      p25_util = ("annual_util_mth_end_bal", lambda x: np.percentile(x, q=25)),
      p50_util = ("annual_util_mth_end_bal", lambda x: np.percentile(x, q=50)),
      p75_util = ("annual_util_mth_end_bal", lambda x: np.percentile(x, q=75)),
      p90_util = ("annual_util_mth_end_bal", lambda x: np.percentile(x, q=90)),
      p99_util = ("annual_util_mth_end_bal", lambda x: np.percentile(x, q=99)),
    )
).reset_index().to_clipboard()

### CL draw type (Cash / Retail spending)

In [18]:
focus_cube.columns

Index(['cc_cst_real', 'uniq_key', 'revenue', 'cost', 'cashback', 'cof',
       'fixed_cost', 'fraud_loss', 'reward_burn', 'scheme_fee', 'annual_fee',
       'cash_adv_fee', 'collection_fee', 'fx_fee', 'interchg_fee', 'int_inc',
       'mftp_total', 'activity_based_costing_expense', 'debt_collection_exp',
       'fee_waived_off', 'marketing_exp', 'specific_business_tax',
       'fee_paid_visa', 'issuer_license_fee', 'total_account_card_fee',
       'total_concierge_fee', 'total_issuer_domestic', 'total_issuer_inter',
       'gain_and_loss', 'interchange_fee', 'merchant_discount_fee',
       'net_interest_rec', 'past_due_amt', 'stmt_amt', 'mth_end_bal',
       'crn_pymt_due_amt', 'pymt_amt', 'cash_draw', 'smc_draw', 'spd_draw',
       'smp_draw', 'usage', 'ecl_delta', 'profit', 'profit_net_ecl',
       'card_status_flag', 'card_annual_segment_adj', 'card_tp',
       'issur_card_lvl2', 'monthly_avg_os_stmt_amt',
       'monthly_avg_os_mth_end_bal', 'monthly_avg_cl', 'annual_stmt_amt',
   

In [56]:
focus_cube_draw = \
(focus_cube
 .assign(draw_cash = lambda x : x["cash_draw"] + x["smc_draw"])
 .assign(draw_retail = lambda x : x["spd_draw"] + x["smp_draw"])
 .loc[:, ["uniq_key", "Peer", "card_annual_segment_adj", 'draw_cash', 'draw_retail', 'usage']]
).compute()

In [59]:
(focus_cube_draw
 .query("usage > 0")
 .assign(pct_draw_cash = lambda x : x["draw_cash"]/x["usage"])
 .assign(pct_draw_retail = lambda x : x["draw_retail"]/x["usage"])
 .groupby(["Peer", "card_annual_segment_adj"])
 .agg(n_account = ("uniq_key", "count"),
      tt_draw = ("usage", "sum"),
      tt_draw_cash = ("draw_cash","sum"),
      tt_draw_retail = ("draw_retail","sum"),
      # avg_pct_draw_retail = ("pct_draw_retail", "mean"), 
      avg_pct_draw_cash = ("pct_draw_cash", "mean"), 
      mode_pct_draw_cash = ("pct_draw_cash", lambda x: x.astype(float).round(3).value_counts().index[0]), 
      mode_pct_draw_cash_n = ("pct_draw_cash", lambda x: x.astype(float).round(3).value_counts()[0]), 
       p1_pct_draw_cash = ("pct_draw_cash", lambda x: np.percentile(x, q=1)),
      p10_pct_draw_cash = ("pct_draw_cash", lambda x: np.percentile(x, q=10)),
      p25_pct_draw_cash = ("pct_draw_cash", lambda x: np.percentile(x, q=25)),
      p50_pct_draw_cash = ("pct_draw_cash", lambda x: np.percentile(x, q=50)),
      p75_pct_draw_cash = ("pct_draw_cash", lambda x: np.percentile(x, q=75)),
      p90_pct_draw_cash = ("pct_draw_cash", lambda x: np.percentile(x, q=90)),
      p99_pct_draw_cash = ("pct_draw_cash", lambda x: np.percentile(x, q=99)),
    )
).reset_index().to_clipboard()

### Time Revolve & Type Revolve

In [19]:
fy2023.columns

Index(['cc_cst_real', 'uniq_key', 'net_interest_rec', 'mftp_total',
       'annual_fee', 'interchange_fee', 'merchant_discount_fee',
       'cash_adv_fee', 'gain_and_loss', 'collection_fee', 'fee_waived_off',
       'debt_collection_exp', 'fee_paid_visa', 'issuer_license_fee',
       'total_issuer_domestic', 'total_issuer_inter', 'total_account_card_fee',
       'total_concierge_fee', 'cashback', 'loyalty_point_redem',
       'activity_based_costing_expense', 'specific_business_tax', 'fraud_loss',
       'credit_limit_card', 'credit_limit_cust', 'credit_utilization',
       'opn_dt', 'card_status', 'past_due_amt', 'stmt_amt', 'mth_end_bal',
       'crn_pymt_due_amt', 'util', 'cr_lmt_amt', 'acq_cnl', 'lmt_at_app',
       'mnth_cd', 'MOB', 'behv', 'pymt_amt', 'dlq_bck', 'ews', 'b_scor',
       'marketing_exp', 'cash_draw', 'smc_draw', 'spd_draw', 'smp_draw',
       'cc_cst_first_num', 'card_flag', 'int_inc', 'cof', 'usage', 'blc_cd',
       'behv_new', 'interchg_fee', 'fx_fee', 'fixed_co

In [27]:
behv_new_agg = (fy2023
                .assign(usage = lambda x : x["cash_draw"] + x["smc_draw"] + x["spd_draw"] + x["smp_draw"])
                .assign(cash_usage = lambda x : x["cash_draw"] + x["smc_draw"])
                .groupby(["uniq_key","behv_new"])
                .agg(n_mnth = ("mnth_cd", "count"),
                     usage = ("usage", "sum"),
                     cash_usage = ("cash_usage", "sum"))
               ).compute().reset_index()

In [28]:
behv_new_agg.head()

,uniq_key,behv_new,n_mnth,usage,cash_usage
0,265606,Revolver,12,0.00,0.0
1,2188536,Revolver,12,17234.75,0.0
2,1433899,Revolver,12,9856.58,0.0
3,3200555,Revolver,12,66027.25,0.0
4,175631,Revolver,10,110319.21,0.0


In [29]:
behv_new_pv = behv_new_agg.pivot(index="uniq_key", columns="behv_new", values="mnth_cd")

KeyError: 'mnth_cd'

In [ ]:
behv_new_pv = behv_new_agg.pivot(index="uniq_key", columns="behv_new", values=["mnth_cd","usage","cash_usage"])

In [ ]:
behv_new_pv.head()

In [25]:
behv_new_pv.columns = list(map("_".join, behv_new_pv.columns))

In [26]:
behv_new_pv.head()

,mnth_cd_Inactive,mnth_cd_Revolver,mnth_cd_Revolver - Cash,mnth_cd_Transactor,usage_Inactive,usage_Revolver,usage_Revolver - Cash,usage_Transactor,cash_usage_Inactive,cash_usage_Revolver,cash_usage_Revolver - Cash,cash_usage_Transactor
uniq_key,,,,,,,,,,,,
1,12.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,12.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,NaN,12.0,NaN,NaN,NaN,26443.2,NaN,NaN,NaN,0.0,NaN,NaN
4,NaN,NaN,NaN,12.0,NaN,NaN,NaN,362937.41,NaN,NaN,NaN,0.0
5,NaN,NaN,12.0,NaN,NaN,NaN,214322.93,NaN,NaN,NaN,109000.0,NaN


In [62]:
focus_peer = focus_cube.loc[:, ["uniq_key", "Peer"]].compute()

In [63]:
peer_pv = focus_peer.merge(behv_new_pv, on="uniq_key", how="left")

In [64]:
peer_rev = (peer_pv.fillna(0.0).assign(n_rev = lambda x : x["Revolver"] + x["Revolver - Cash"]))

In [65]:
peer_rev.head()

,uniq_key,Peer,Inactive,Revolver,Revolver - Cash,Transactor,n_rev
0,1895412,Affluent Income,0.0,4.0,0.0,0.0,4.0
1,4617220,Mass,0.0,12.0,0.0,0.0,12.0
2,1784924,Mass,0.0,0.0,12.0,0.0,12.0
3,1713133,Affluent Income,3.0,4.0,0.0,5.0,4.0
4,3461612,Mass,0.0,12.0,0.0,0.0,12.0


In [67]:
(peer_rev
 .groupby(["Peer", "n_rev", "Revolver - Cash"], as_index=False)
 .agg(n_account = ("uniq_key", "count"))
).to_clipboard()

In [41]:
(peer_rev
 .assign(pct_cash_rev = lambda x : x["Revolver - Cash"]/x["n_rev"])
 .groupby(["Peer"])
 .agg(n_account = ("uniq_key", "count"),
      avg_n_rev = ("n_rev", "mean"),
      avg_n_cash_rev = ("Revolver - Cash", "mean"),
      
      avg_pct_cash_rev = ("pct_cash_rev", "mean"), 
       p1_pct_cash_rev = ("pct_cash_rev", lambda x: np.percentile(x, q=1)),
      p10_pct_cash_rev = ("pct_cash_rev", lambda x: np.percentile(x, q=10)),
      p25_pct_cash_rev = ("pct_cash_rev", lambda x: np.percentile(x, q=25)),
      p50_pct_cash_rev = ("pct_cash_rev", lambda x: np.percentile(x, q=50)),
      p75_pct_cash_rev = ("pct_cash_rev", lambda x: np.percentile(x, q=75)),
      p90_pct_cash_rev = ("pct_cash_rev", lambda x: np.percentile(x, q=90)),
      p99_pct_cash_rev = ("pct_cash_rev", lambda x: np.percentile(x, q=99)),
    )
).to_clipboard()

### Profit distribution

In [42]:
focus_cube_profit = focus_cube.loc[:, ["uniq_key", "Peer", 'ecl_delta', 'profit', 'profit_net_ecl']].compute()

In [73]:
with pd.option_context("display.float_format", "{:,.2f}".format, "display.max_columns", None, 'display.max_colwidth', None):
    display(focus_cube_profit["profit_net_ecl"].describe())

count      600,898.00
mean         5,725.85
std         19,344.47
min     -2,130,448.64
25%            146.60
50%          2,163.62
75%          6,572.09
max      1,642,135.12
Name: profit_net_ecl, dtype: float64

In [88]:
# np.arange(-100000,105000,5000)

profit_range = [-np.inf] + np.arange(-100000,105000,5000).tolist() + [np.Inf]
profit_range
(focus_cube_profit
 .assign(profit_gr = lambda x : pd.cut(x["profit_net_ecl"], bins=profit_range))
 .groupby(["Peer", "profit_gr"])
 .agg(n_card = ("uniq_key", "count"),
      n_profit_net_ecl = ("profit_net_ecl", "sum"))
).reset_index().to_clipboard()